## 6.4 LSTM을 사용한 언어 모델

In [ ]:
import sys
sys.path.append('..')
from common.time_layers import TimeEmbedding, TimeLSTM, TimeAffine, TimeSoftmaxWithLoss
import pickle
import numpy as np
from numpy.random import randn as rn
from data_set import ptb
from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from common.utill import eval_perplexity

class Rnnlm:
    def __init__(self, vocab_size=10000, wordvec_size=100, hidden_size=100):
        V, D, H = vocab_size, wordvec_size, hidden_size

        # 가중치 초기화
        embed_W = (rn(V, D) / 100).astype(np.float64) # 전체 단어 수 에서 압축 할 차원수로 가기 위한 Weight
        lstm_Wx = (rn(D, 4*H)/np.sqrt(D)).astype(np.float64) # Embedding된 Vector 차원 수 => hidden state 4개(f, i, o, g)로 되기 위한 Weight
        lstm_Wh = (rn(H, 4*H)/np.sqrt(H)).astype(np.float64) # Hidden State State => 4개의 Hidden state 가 되기 위한 Weight 
        lstm_b = np.zeros(4 * H).astype(np.float64) # W들 연산 이후 더해질 Bias
        affine_W = (rn(H, V)) # LSTM 각 layer에서 나온 Output의 Affine 연산을 한번에 계산하기 위한 TimeAffine Layer의 Weight Hidden state 수 => 분류할 총 vocab 크기
        affine_b = np.zeros(V).astype(np.float64)

        # 계층 생성
        self.layers = [
            TimeEmbedding(embed_W),
            TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.lstm_layer = self.layers[1]

        # 모든 가중치와 기울기를 리스트에 모은다.
        self.params = []
        self.grads = []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def predict(self, xs):
        for layer in self.layers:
            xs = layer.forward(xs)
        return xs
    
    def forward(self, xs, ts):
        score = self.predict(xs)
        loss = self.loss_layer.forward(score, ts)
        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
    
    def reset_state(self):
        self.lstm_layer.reset_state()
    
    def save_params(self, file_name='Rnnlm.pkl'):
        with open(file_name, 'wb') as f:
            pickle.dumps(self.params, f)
    
    def load_params(self, file_name='Rnnlm.pkl'):
        with open(file_name, 'rb') as f:
            self.params = pickle.load(f)


# ===== 학습 =====

# 하이퍼파라미터
batch_size = 20
wordvec_size = 100
hidden_size = 100
time_size = 35
lr = 20.0
max_epoch = 4
max_grad = 0.25

# 학습데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_test, _, _ = ptb.load_data('test')
vocab_size = len(word_to_id)
xs = corpus[:-1]
tx = corpus[1:]

# 모델 생성
model = Rnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)
trainer.fit(xs, tx, max_epoch, batch_size, time_size, max_grad, eval_interval=20)
trainer.plot(ylim=(0, 500))

model.reset_state()
ppl_test = eval_perplexity(model, corpus_test)
print(f"Perflexity Test: {ppl_test}")

model.save_params()

#

## 6.5 RNNLM 추가 개선

- LSTM 계층 다층화
- Dropout을 통한 Overfitting 방지
- Weight Tying을 통한 학습 매개변수 감소 및 정확도 증가

#### 6.5.1 LSTM 계층 다층화

![image](https://github.com/choibigo/Study/assets/38881179/d308c8dd-5f01-4aec-a103-8a965c408f70)

- LSTM을 겹쳐서 사용하면 언어의 정확도가 향샹된다.
- 1번쨰 LSTM 계층의 Hidden state가 2번째 계층의 Input으로 입력된다.

#### 6.5.2 드롭아웃에 의한 과적합 억제
- LSTM 계층을 다층화 하면 시계열 데이터의 복잡한 의존 관계를 학습할 수 있을것이라 기대한다.
- 층을 깊게 함으로써 표현력이 풍부한 모델을 만들 수 있다.
- 그러나 'Over fitting'문제가 발생할 수 있다.
- RNN은 일반적인 피드포워드 신경망보다 쉽게 과적합을 일으킬 수 있다.
- 따라서 RNN에도 Regularization, Normalize, Dropout과 같은 과적합 대책이 중요하다

![image](https://github.com/choibigo/Study/assets/38881179/b0f3af2b-d5b8-4a88-945b-68dcccaf2b07)

- 시계열 방향으로 Dropout Layer를 삽입하는 것은 좋은 방법이 아니다.
- 시간에 흐름에 따라 정보가 사라질 수 있다. 즉, 시간에 비례해 드롭아웃에 의한 노이즈가 축적 된다.
- 시간에 흐름에 따른 기억 손실을 예방하려고 Cell을 만들었는데 Dropout을 적용해 버리면 소용없어 진다.

![image](https://github.com/choibigo/Study/assets/38881179/3cc04141-7a33-4578-96bc-dc8cb5a105b0)

- Dropout Layer를 깊이 방향 으로 삽입할 수 도 있다.
- 이렇게 구성하면 시간 방향으로 아무리 진행해도 정보를 잃지 않는다.
- 드롭아웃이 시간축과 독립적으로 깊이 방향으로만 영향을 준다.

![image](https://github.com/choibigo/Study/assets/38881179/ed716177-6a17-43f1-a796-118f8b659589)

- 일반저깅 Dropout은 시간 방향에는 적합하지 않다.
- 그러나 최근 연구에서는 RNN의 시간 방향 정규화를 목표로하는 다양한 방법이 제안된다.
- Variational Dropout을 통해 시간 방향으로 Dropout을 적용하는데 성공했다.
- Variational Dropout은 깊이 방향과 시간 방향에도 이용할 수 있어 언어 모델의 정확도를 향상시킬 수 있다.
- 같은 계층에 속한 Dropout들은 같은 Mask를 공유한다, Mask는 데이터의 통과/차단을 결정하는 Binary 형태의 무작위 패턴이다.
- 위 그림에서 색이 같은 Dropout 계층 들은 고정된 Mask를 이용한다.
- 마스크를 고정함으로써 정보를 잃게 되는 방법도 '고정'되므로, 일반적인 드롭아웃 때와 달리 정보가 지수적으로 손실되는 것을 방지할 수 있다.

#### 6.5.3 가중치 공유
- 언어 모델을 개선하는 간단한 트릭은 'Weight tying이 있다.

![image](https://github.com/choibigo/Study/assets/38881179/5cff8d01-a854-42d6-901a-4dfbfa367062)

- Embedding 계층의 가중치와 Affine 계층의 가중치를 연결하는 기법이 가중치 공유(Weight Tying)이다.
- 가중치를 공유함으로써 매개변수 수를 줄일 수 있다 이에 따라 학습이 쉬워져 정확도도 향상되는 기법이다.
- 어휘수를 V Hidden sTate의 수를 H라고 했을 때, Embedding 계층의 가중치는 V x H로 나타낼 수 있다, Affine 계층을 H x v 이다.
- Emgedding 계층의 가중치를 전치하여 Affine 계층의 가중치로 설정할 수 있다.

#### 6.5.4 개선된 RNNLM 구현

![image](https://github.com/choibigo/Study/assets/38881179/9405f74e-5c2a-4172-a2c0-fe7ea274beac)

In [5]:
import sys
sys.path.append('..')
from common.time_layers import *
import numpy as np
from common.base_model import BaseModel
from numpy.random import randn as rn


class BetterRnnlm(BaseModel):
    def __init__(self, vocab_size=10000, wordvec_size=650, hidden_size=650, dropout_ratio=0.5):
        V, D, H = vocab_size, wordvec_size, hidden_size
        
        embed_W = (rn(V, D) / 100).astype(np.float64)
        lstm_Wx1 = (rn(D, 4*H) / np.sqrt(D)).astype(np.float64)
        lstm_Wh1 = (rn(H, 4*H) / np.sqrt(H)).astype(np.float64)
        lstm_b1 = np.zeros(4*H).astype(np.float64)
        lstm_Wx2 = (rn(D, 4*H) / np.sqrt(D)).astype(np.float64)
        lstm_Wh2 = (rn(H, 4*H) / np.sqrt(H)).astype(np.float64)
        lstm_b2 = np.zeros(4*H).astype(np.float64)
        affine_b = np.zeros(V).astype(np.float64)

        self.layers = [
            TimeEmbedding(embed_W),
            TimeDropout(dropout_ratio),
            TimeLSTM(lstm_Wx1, lstm_Wh1, lstm_b1, stateful=True),
            TimeDropout(dropout_ratio),
            TimeLSTM(lstm_Wx2, lstm_Wh2, lstm_b2, stateful=True),
            TimeDropout(dropout_ratio),
            TimeAffine(embed_W.T, affine_b)
        ] 
        # dropout을 각 layer 마다 추가함
        # Affine과 Embedding Weight를 공유함
        # LSTM계층을 깊게 쌓음

        self.loss_layer = TimeSoftmaxWithLoss()
        self.lstm_layers = [self.layers[2], self.layers[4]]
        self.drop_layers = [self.layers[1], self.layers[3], self.layers[5]]
        
        self.params = []
        self.grads = []

        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def predict(self, xs, train_flg=False):
        for layer in self.drop_layers:
            layer.train_flg = train_flg
        for layer in self.layers:
            xs = layer.forward(xs)

        return xs

    def forward(self, xs, ts, train_flg=True):
        score = self.predict(xs, train_flg)
        loss = self.loss_layer.forward(score, ts)
        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
    
    def reset_state(self):
        for layer in self.lstm_layers:
            layer.reset_state()

In [ ]:
import sys
sys.path.append('..')
from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from common.utill import eval_perplexity
from data_set import ptb

#Hyper Parameter
batch_size = 20
wordvec_size = 650
hidden_size = 650
time_size = 35
lr = 20.0
max_epoch = 40
max_grad = 0.25
dropout_ratio = 0.5

# Data Load
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_val, _, _ = ptb.load_data('val')
corpus_test, _, _ = ptb.load_data('test')

vocab_size = len(word_to_id)
xs = corpus[:-1]
tx = corpus[1:]

model = BetterRnnlm(vocab_size, wordvec_size, hidden_size, dropout_ratio)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

# Early Stop & Scheduling
best_ppl = float('inf')
for epoch in range(max_epoch):
    trainer.fit(xs, tx, max_epoch=1, batch_size=batch_size, time_size=time_size, max_grad=max_grad)

    model.reset_state()
    ppl = eval_perplexity(model, corpus_val)
    print(f"Perplexity : {ppl}")

    if best_ppl > ppl:
        best_ppl = ppl
        model.save_params()
    else:
        lr /= 4.0
        optimizer.lr = lr
    
    model.reset_state()
    print('='*50)